# Modelling Data Downloading & Preparation

This notebook focusses on the remaining dimensions used by the Random Forest to learn about, and make predictions of, the scores. Where data can be downloaded automatically this notebook will do so. Where it cannot (hello Nomis and your incomprehensible API and broken API tool!) then I have made it obvious from which Census table the data were taken and how the downloaded data were processed.

In [ ]:
import matplotlib as mpl
mpl.use('TkAgg')
%matplotlib inline
import matplotlib.pyplot as plt 

In [ ]:
# For reproducibility
import random
import numpy as np
r_state = 42
random.seed(r_state) 
np.random.seed(r_state)

In [ ]:
import pandas as pd
import geopandas as gpd
import requests
import glob
import re
import os
import io
import zipfile
from io import BytesIO

from geoconvert import geoconvert

lkp = os.path.join('data','lkp')
src = os.path.join('data','src')

canonical  = os.path.join('data','canonical')
converted  = os.path.join(canonical,'converted')
greenspace = os.path.join(canonical,'greenspace')
dwelling   = os.path.join(canonical,'dwellings')
travel     = os.path.join(canonical,'travel')
household  = os.path.join(canonical,'households')
housing    = os.path.join(canonical,'housing')
work       = os.path.join(canonical,'work')

for d in [canonical,converted,greenspace,dwelling,travel,household,housing,work]:
    if not os.path.exists(d):
        os.makedirs(d)

In [ ]:
# Make sure you always run this!
boroughs = ['City of London','Barking and Dagenham','Barnet','Bexley','Brent','Bromley',
            'Camden','Croydon','Ealing','Enfield','Greenwich','Hackney','Hammersmith and Fulham',
            'Haringey','Harrow','Havering','Hillingdon','Hounslow','Islington',
            'Kensington and Chelsea','Kingston upon Thames','Lambeth','Lewisham',
            'Merton','Newham','Redbridge','Richmond upon Thames','Southwark','Sutton',
            'Tower Hamlets','Waltham Forest','Wandsworth','Westminster']

In [ ]:
ldn2011 = pd.read_pickle(os.path.join(lkp,'LSOAs 2011.pkl'))
ldn2004 = pd.read_pickle(os.path.join(lkp,'LSOAs 2004.pkl'))

print("Have built London LSOA filter data for use where needed...")
print("\t2001: " + str(ldn2004.shape[0]) + " rows.")
print("\t2011: " + str(ldn2011.shape[0]) + " rows.")

In [ ]:
def convert_to_2011(df,src,dest,nm):
    gc = geoconvert.geoconvert()
    gc.auto_2001_to_2011(os.path.join(src,nm))

    for f in glob.glob(re.sub("-\d+\.csv","*",nm)):
        fn = re.sub("-converted","",f)
        print("Moving " + f + " to " + converted)
        os.rename(f, os.path.join(converted,fn))
    
    dfc = pd.read_csv(os.path.join(converted,nm), index_col=False)
    
    dfc.columns = df.columns
    
    dfc.to_csv(os.path.join(dest,nm), index=False)
    print("\tConverted file has " + str(dfc.shape[0]) + " rows.")
    print(dfc.sample(2, random_state=r_state))

## Ward to LSOA Conversion (Greenspace)

For some strange reason access to greenspace has only been tracked at the [Ward level](https://data.london.gov.uk/dataset/access-public-open-space-and-nature-ward) using a mix of 2013 and 2014 ward codes spread across _two_ sheets in an XLSX so we need to combine them and convert to some kind of LSOA score using the [ONS lookup](http://geoportal.statistics.gov.uk/datasets/lower-layer-super-output-area-2011-to-ward-2015-lookup-in-england-and-wales). I have tried using the 2011 LSOA-to-2015 Ward lookup table but end up missing out on data for Hackney, Tower Hamlets, and Kensington & Chelsea so the `_JR` file from the .

In [ ]:
lsoa2ward  = pd.read_csv(os.path.join(lkp,'LSOA_WARD_JR.csv'))

print("There are " + str(lsoa2ward.gss_cd.unique().shape[0]) + " wards.")
print("There are " + str(lsoa2ward.shape[0]) + " 2011 LSOAs.")
lsoa2ward.sample(3, random_state=r_state)

In [ ]:
url = ('https://files.datapress.com/london/dataset/'
       'access-public-open-space-and-nature-ward/'
       'access-public-open-space-ward.xls')

# Retrieve it
greensp1 = pd.read_excel(url, sheet_name='Open space 2013 wards')

# Rename columns
greensp1.rename(columns={
    'Ward_GSS_CODE':'wardgsscd',
    'BOROUGH_Name':'Borough',
    '% Open Space with access':'% open space with access'
}, inplace=True)

greensp1 = greensp1[['wardgsscd','% open space','% open space with access','% of open space that has access']]

### Now the renamed wards...
greensp2 = pd.read_excel(url, sheet_name='Open space 2014 wards')

# Rename columns
greensp2.rename(columns={
    'Ward_GSS_CODE':'wardgsscd',
    'BOROUGH_Name':'Borough',
    'Percentage_OpenSpace_Area_ALL_GIGL_open_space':'% open space',
    'Percent_OpenSpace_Area_GIGLdesignatedsitesWithAccess':'% open space with access'
}, inplace=True)

greensp2 = greensp2[['wardgsscd','% open space','% open space with access','% of open space that has access']]

greensp = pd.concat([greensp1, greensp2], ignore_index=True)

# Sanity check
print("Combined we have " + str(greensp.shape[0]) + " rows.")
greensp.sample(3, random_state=r_state)

In [ ]:
# Inner join should yield about 4.8k matches as LSOAs smaller than wards
lsoagrn = pd.merge(greensp, lsoa2ward, how='inner', left_on='wardgsscd', right_on='gss_cd')

# Drop the bits we're not interested in
lsoagrn = lsoagrn[['lsoacd','% open space','% open space with access','% of open space that has access']]

# Save it
lsoagrn.to_csv(os.path.join(greenspace,'Share.csv'), index=False)

print("Matching rows: " + str(lsoagrn.shape[0]))
lsoagrn.sample(3, random_state=r_state)

## Ward to LSOA Conversion (Greenspace Access)

For some strange reason access to greenspace has only been tracked at the [Ward level](https://data.london.gov.uk/dataset/access-public-open-space-and-nature-ward) using 2013 and 2014 ward codes so we need to convert it to some kind of LSOA score using the [ONS lookup](http://geoportal.statistics.gov.uk/datasets/lower-layer-super-output-area-2011-to-ward-2015-lookup-in-england-and-wales). I have tried using the [2011 LSOA-to-2015 Ward lookup table](https://opendata.arcgis.com/datasets/07a6d14d4a0540769f0662f4d1450bae_0.csv) from the ONS but end up missing out on data for Hackney, Tower Hamlets, and Kensington & Chelsea because it doesn't have the 2014 boundary changes or something.

In [ ]:
lsoa2ward  = pd.read_csv(os.path.join(lkp,'LSOA_WARD_JR.csv'))

url = ('https://files.datapress.com/london/dataset/'
       'access-public-open-space-and-nature-ward/'
       'access-public-open-space-ward.xls')

# Retrieve it
greenspacc1 = pd.read_excel(url, sheet_name='Access to open space 2013 wards', header=1)

# Rename columns to be GIS-friendly
greenspacc1.rename(columns={
    'WD13CD':'gss_cd',
    'Borough name':'Borough'
}, inplace=True)

greenspacc1 = greenspacc1[['gss_cd','Open Space','Local Parks','District Parks','Metropolitan Parks','Regional Parks']]
print("Have " + str(greenspacc1.shape[0]) + " rows from 2013 wards.")
#greenspacc1.sample(3, random_state=r_state)

### Now the renamed wards...
greenspacc2 = pd.read_excel(url, sheet_name='Access to open space 2014 wards', header=1)

# Rename columns to be GIS-friendly
greenspacc2.rename(columns={
    'WD13CD':'gss_cd',
    'Borough name':'Borough'
}, inplace=True)

greenspacc2 = greenspacc2[['gss_cd','Open Space','Local Parks','District Parks','Metropolitan Parks','Regional Parks']]
print("Have " + str(greenspacc2.shape[0]) + " rows from 2014 wards.")
#greenspacc2.sample(3, random_state=r_state)

greenspacc = pd.concat([greenspacc1, greenspacc2], ignore_index=True)

# Sanity check
print("Combined we have " + str(greenspacc.shape[0]) + " rows.")

In [ ]:
# Inner join should yield about 4.8k matches as LSOAs smaller than wards
#test = pd.merge(green, lsoa2ward, how='inner', left_on='Ward name', right_on='WD15NM')
lsoagacc = pd.merge(greenspacc, lsoa2ward, how='inner', left_on='gss_cd', right_on='gss_cd')

# Duplicates in these sheets for some reason (this is poorly-managed data!)
#lsoagacc[lsoagacc.duplicated(subset='lsoa11cd', keep='first')].index.values
lsoagacc = lsoagacc.drop(lsoagacc[lsoagacc.duplicated(subset='lsoacd', keep='first')].index.values)

# Drop the bits we're not interested in
lsoagacc = lsoagacc[['lsoacd','Open Space','Local Parks','District Parks','Metropolitan Parks','Regional Parks']]

# Save it
lsoagacc.to_csv(os.path.join(greenspace,'Access.csv'), index=False)

# Sanity check
print("Matching rows: " + str(lsoagacc.shape[0]))
lsoagacc.sample(3, random_state=r_state)

## Dwelling Period by LSOA

In [ ]:
url = ('https://files.datapress.com/london/dataset/'
       'property-build-period-lsoa/'
       'dwelling-period-built-2014-lsoa.csv')

# Note suppressed values
age = pd.read_csv(url, index_col=False, na_values="-")
age.rename(columns={'lsoa':'lsoacd'}, inplace=True)
age.drop(['GEOG','Name'], axis=1, inplace=True)

# Not formatted as numeric
for c in age.columns[3:]:
    age[c] = pd.to_numeric(age[c].str.replace(",",""))

# Sanity check
print("Have " + str(age.shape[0]) + " rows data.")
age.head(3)

In [ ]:
url = ('https://files.datapress.com/london/dataset/'
       'property-build-period-lsoa/'
       'dwelling-period-built-2015-lsoa-msoa.csv')

# Note suppressed values
age = pd.read_csv(url, index_col=False, low_memory=False, na_values="-")
age.rename(columns={'ECODE':'lsoacd', 'ALL_PROPERTIES':'total'}, inplace=True)

age = age.loc[age.BAND=='All',:]
age.drop(['GEOGRAPHY','AREA_NAME','BAND'], axis=1, inplace=True)

# Not formatted as numeric
for c in age.columns[1:]:
    age[c] = pd.to_numeric(age[c].str.replace(",",""))

# Sanity check
print("Have " + str(age.shape[0]) + " rows data.")
age.head(3)

In [ ]:
# Drop the non-London LSOAs
ldn_age  = age.loc[age.lsoacd.isin(ldn2011.lsoacd.values)].copy()

# Rename columns and set index
ldn_age.set_index('lsoacd', inplace=True)
ldn_age.rename(columns=lambda x:x.replace('_','-').replace('BP-','Build Period: '), inplace=True)

# I did experiment with trying to calculate a per annum 
# construction rate in the period prior to each Census
# but the time periods are a bit arbitrary and so it's
# difficult to line them up in a useful way
ldn_age.drop(['Build Period: 2000-2009','Build Period: 2010-2015'], inplace=True, axis=1)
ldn_age.fillna(0, inplace=True)
#recent = pd.DataFrame()
#recent['pre_2001pa'] = age['1993_1999'].apply(lambda x: np.around(x/7.0))  # 7 years in data
#recent['pre_2011pa'] = age['2000_2009'].apply(lambda x: x/10.0) # 10 years in data
#recent['pre_2021pa'] = age['2010_2014'].apply(lambda x: x/5.0)  # 5 years in data
#ldn_age = ldn_age.reset_index().merge(recent, how='left', left_on='lsoacd', right_index=True).fillna(0).set_index('lsoacd')

ldn_age.to_csv(os.path.join(dwelling,'Age.csv'), index=True)

# Sanity check
print("Have " + str(ldn_age.shape[0]) + " rows data.")
print("Done.")

## Travel Time to Major Infrastructure

<p style="color:red;font-weight:bold">**Do not run this block on a limited connection or one where you are charged per MB or GB.**</p>

In [ ]:
# This data is part of a big zipfile, don't 
# download on a billable connection
import os, io, requests, zipfile, glob
from io import BytesIO

if not os.path.exists(os.path.join(src,'tmp')):
    os.makedirs(os.path.join(src,'tmp'))

# Where to get it, and then how we extract it...
url = ('https://www.gov.uk/government/uploads/system/uploads/'
       'attachment_data/file/318920/'
       'connectivity-statistics.zip')

print("Downloading...")
r = requests.get(url, stream=True)
z = zipfile.ZipFile(BytesIO(r.content))
print("Extracting...")
z.extractall(os.path.join(src,'tmp'))
print("Done.")

In [ ]:
# We're only interested in the con0111 file
connect = pd.read_excel(glob.glob(os.path.join(src,'tmp','*con0111*'))[0], 
                        sheet_name='CON0111b_Selected', header=7, skipfooter=28, na_values="..")

# This is a huge data set, so let's drop the 
# irrelevant data as quickly as possible
s11 = set(ldn2011.lsoacd.values)
s04 = set(ldn2004.lsoacd.values)

connect = connect[connect['LSOA Code'].isin(list(s11.union(s04)))]

# Tidy up
connect.drop(['Upper tier local authority code',
              'Upper tier local authority name',
              'LSOA population (2011)',
              'LSOA Name',
              'Unnamed: 5'
             ], axis=1, inplace=True)

# Rename column
connect.rename(columns={
    'LSOA Code':'lsoacd'
}, inplace=True)

# And set as index
connect.set_index('lsoacd', inplace=True) # 'LSOA04CD', inplace=True)

# Let's focus on airports
connect = connect.loc[:, [re.search("(?:Heathrow|London|Gatwick|Stansted)",x) is not None for x in connect.columns] ]
connect = connect.replace('..',np.NaN) # This should have happened above... but just in case

# Sanity check, should be 4765
print("Have " + str(connect.shape[0]) + " rows of data.")
connect.sample(3, random_state=r_state)

In [ ]:
# The first set of columns above is for public transit travel time
pub_trans = connect[[col for col in connect.columns if '.1' not in col]]

# Find the columns to do with Heathrow (LHR) and Gatwick (LGW)
lhr = tuple([col for col in pub_trans.columns if 'Heathrow' in col])
lgw = tuple([col for col in pub_trans.columns if 'Gatwick' in col])

# Calculate mean travel time to the airport as a whole
pub_trans = pub_trans.assign(Heathrow=pub_trans.loc[:, lhr].mean(axis=1))
pub_trans = pub_trans.assign(Gatwick=pub_trans.loc[:, lgw].mean(axis=1))

# Drop the terminal-specific columns
pub_trans = pub_trans.loc[:, [re.search("(?:T\d|Terminal)",x) is None for x in pub_trans.columns]]

# Save
pub_trans.to_csv(os.path.join(travel,'Infrastructure Access-Public Transit.csv'), index=True)

# Sanity check
print("Have " + str(pub_trans.shape[0]) + " rows data.")
pub_trans.sample(3, random_state=r_state)

In [ ]:
# The second set of columns above is for private vehicle travel time
pri_trans = connect[[col for col in connect.columns if '.1' in col]]

# Find the columns to do with Heathrow (LHR) and Gatwick (LGW)
lhr = tuple([col for col in pri_trans.columns if 'Heathrow' in col])
lgw = tuple([col for col in pri_trans.columns if 'Gatwick' in col])

# Calculate mean travel time to the airport as a whole
pri_trans = pri_trans.assign(Heathrow=pri_trans.loc[:, lhr].mean(axis=1))
pri_trans = pri_trans.assign(Gatwick=pri_trans.loc[:, lgw].mean(axis=1))

# Drop the terminal-specific columns -- this is 
# probably where the set-by-copy warning is coming
# from and could be fixed by taking the inverse of 
# the re.search result and adding an inplace=True to
# a drop command.
pri_trans = pri_trans.loc[:, [re.search("(?:T\d|Terminal)",x) is None for x in pri_trans.columns]].copy()

# Tidy up the column names
pri_trans.rename(columns=lambda x: x.replace(".1",""), inplace=True)

# Save
pri_trans.to_csv(os.path.join(travel,'Infrastructure Access-Private Vehicle.csv'), index=True)

# Sanity check
print("Have " + str(pri_trans.shape[0]) + " rows data.")
pri_trans.sample(3, random_state=r_state)

In [ ]:
import shutil 
import os
shutil.rmtree(os.path.join(src,'tmp'))
print("Done.")

## Travel time to Bank Station

This data is provided by the 'My London' service from the London Data Store (usefully demonstrated here: [My London](http://my.london.gov.uk/)). It would be nice if this were properly time-stamped so that we could compare travel times in 2001 with travel times now, but this sort of thing appears not to be on the GLA's or TfL's radar.

In [ ]:
url = ('https://files.datapress.com/london/dataset/'
       'mylondon/MyLondon_traveltime_to_Bank_station_OA.csv')

ttb  = pd.read_csv(url)

# Add on LSOA data
lsoa2oa = pd.read_csv(os.path.join(lkp,'LSOA_OA_JR.csv'))
ttb = pd.merge(lsoa2oa, ttb, left_on='oacd', right_on='OA11CD', how='inner')

# Tidy up
ttb.drop(['oacd','OA11CD'], axis=1, inplace=True)

# Calculate mean travel time for the LSOA from all OAs
ttb_lsoa = ttb.groupby('lsoacd').mean()

# Save it
ttb_lsoa.to_csv(os.path.join(travel,'Travel Time To Bank.csv'), index=True)

# Sanity check
print("Have " + str(ttb_lsoa.shape[0]) + " rows of data.")
print("Done.")
ttb_lsoa.sample(3, random_state=r_state)

## My Fare Zone

From the same My London service is one that tells you the Fare Zone for any Output Area in London. We take the _average_ Fare Zone for each LSOA since that is still a reasonable representation of the cost of travel for that LSOA. An alternative would be to take the mode:
```python
mfz.groupby('lsoacd')['Fare_Zone'].agg(lambda x:x.value_counts().index[0])
```

In [ ]:
url = ('https://files.datapress.com/london/dataset/'
       'mylondon/MyLondon_fare_zone_OA.csv')

mfz  = pd.read_csv(url)

# Add on LSOA data
mfz = pd.merge(lsoa2oa, mfz, left_on='oacd', right_on='OA11CD', how='inner')

# Tidy up
mfz.drop(['oacd','OA11CD'], axis=1, inplace=True)

# Calculate mean travel time for the LSOA from all OAs
# -- nearly a pointless exercise, but useful to know if
# LSOAs straddle a fare boundary and could give us some
# handy 'interpolation'
mfz_lsoa = mfz.groupby('lsoacd').mean()

# Save it
mfz_lsoa.to_csv(os.path.join(travel,'Fare Zone.csv'), index=True)

# Sanity check
print("Have " + str(mfz_lsoa.shape[0]) + " rows data.")
print("Done.")
mfz_lsoa.sample(3, random_state=r_state)

## Accommodation Mix

Tables:
- UV042
- QS401EW

In [ ]:
dtype = pd.read_csv(os.path.join(src,'2001','uv042.csv.gz'),
    header=5, skipfooter=4, skip_blank_lines=True, engine='python', compression='gzip')

# Convert the columns names to something more tractable -- 
# note the very slight (but annoying differences) between 
# the two Census years.
dtype.rename(columns={
    'mnemonic':'lsoacd',
    'super output areas - lower layer':'lsoanm',
    'All categories: Accommodation type':'total'
}, inplace=True)
dtype.rename(columns=lambda x: re.sub('^Unshared dwelling(?:\: Whole house or bungalow|\: Flat, maisonette or apartment)?: ','',x), inplace=True)
dtype.rename(columns=lambda x: re.sub(' \(.+?\)','',x), inplace=True)

# Drop the non-London LSOAs
dtype.drop(['lsoanm'], axis=1, inplace=True)
dtype = dtype[dtype.lsoacd.isin(ldn2004.lsoacd.values)]

# And save
dtype.to_csv(os.path.join(src,'Type-2001.csv'), index=False)

# convert_to_2011(df,src,dest,nm)
convert_to_2011(dtype, src, dwelling, 'Type-2001.csv')

In [ ]:
dtype2011 = pd.read_csv(os.path.join(src,'2011','qs401ew.csv.gz'), 
                      skip_blank_lines=True, header=6, skipfooter=5, engine='python', compression='gzip')

# Convert the column names to something more tractable
dtype2011.rename(columns={
    'mnemonic':'lsoacd',
    '2011 super output area - lower layer':'lsoanm',
    'All categories: Accommodation type':'total'
}, inplace=True)
dtype2011.rename(columns=lambda x: re.sub('^Unshared dwelling(?:\: Whole house or bungalow|\: Flat, maisonette or apartment)?: ','',x), inplace=True)
dtype2011.rename(columns=lambda x: re.sub(' \(.+?\)','',x), inplace=True)

# Drop the non-London LSOAs
dtype2011.drop(['lsoanm'], axis=1, inplace=True)
dtype2011 = dtype2011[dtype2011.lsoacd.isin(ldn2011.lsoacd.values)]

# And save
dtype2011.to_csv(os.path.join(dwelling,'Type-2011.csv'), index=False)

# Sanity check
print("2011 Accommodation Type data frame contains " + str(dtype2011.shape[0]) + " rows.")
dtype2011.sample(3, random_state=r_state)

## Travel Mode

Tables:
- UV039 
- QS701EW

Note that the ONS [points out](https://www.nomisweb.co.uk/census/2011/qs701ew) that is not advisable to directly compare these two data sets because of changes in the way that home-workers are counted. Unfortunately, their suggested Custom Table (CT0015) alternative only has LA-level data which is rather unhelpful. My hope is that by focussing on percentage share, not raw numbers we can control for this to some extent. As well, since all LSOAs should be affected in the same way this _may_ not be as much of a problem.

In [ ]:
ttw2001 = pd.read_csv(os.path.join(src,'2001','uv039.csv.gz'),
    header=5, skipfooter=4, skip_blank_lines=True, engine='python', compression='gzip')

# Convert the columns names to something more tractable -- 
# note the very slight (but annoying differences) between 
# the two Census years.
ttw2001.rename(columns=lambda x: re.sub("; measures: Value","",re.sub("Method of travel to work: ","",x)), inplace=True)
ttw2001.rename(columns={
    'mnemonic':'lsoacd',
    'super output areas - lower layer':'lsoanm',
    'All categories: Method of travel to work':'Total',
    'Work mainly at or from home':'Homeworker', 
    'Underground, metro, light rail or tram':'Tube or Tram',
    'Train':'Train', 
    'Bus, minibus or coach':'Bus',
    'Motorcycle, scooter or moped':'Motorcycle or Moped',
    'Driving a car or van':'Private Vehicle',
    'Passenger in a car or van':'Passenger in Private Vehicle',
    'Taxi or minicab':'Taxi',
    'Bicycle':'Bicycle',
    'On foot':'Foot',
    'Other method of travel to work':'Other travel method',
    'Not currently working':'Not in employment'
}, inplace=True)

# Drop the non-London LSOAs
ttw2001.drop(['lsoanm'], axis=1, inplace=True)
ttw2001 = ttw2001[ttw2001.lsoacd.isin(ldn2004.lsoacd.values)]

# And save
ttw2001.to_csv(os.path.join(src,'TTW-2001.csv'), index=False)

# convert_to_2011(df,src,dest,nm)
convert_to_2011(ttw2001, src, travel, 'TTW-2001.csv')

In [ ]:
# Using the QS703EW table...
ttw2011 = pd.read_csv(os.path.join(src,'2011','qs701ew.csv.gz'), 
                      skip_blank_lines=True, header=6, skipfooter=5, engine='python', compression='gzip')

# Convert the column names to something more tractable
ttw2011.rename(columns=lambda x: re.sub("; measures: Value","",re.sub("Method of Travel to Work: ","",x)), inplace=True)
ttw2011.rename(columns={
    'mnemonic':'lsoacd',
    '2011 super output area - lower layer':'lsoanm',
    'All categories: Method of travel to work':'Total',
    'Work mainly at or from home':'Homeworker', 
    'Underground, metro, light rail, tram':'Tube or Tram',
    'Train':'Train', 
    'Bus, minibus or coach':'Bus',
    'Motorcycle, scooter or moped':'Motorcycle or Moped',
    'Driving a car or van':'Private Vehicle',
    'Passenger in a car or van':'Passenger in Private Vehicle',
    'Bicycle':'Bicycle',
    'On foot':'Foot',
    'Other method of travel to work':'Other travel method'
}, inplace=True)

ttw2011.sample(3, random_state=r_state)

# Drop the non-London LSOAs
ttw2011.drop(['lsoanm'], axis=1, inplace=True)
ttw2011 = ttw2011[ttw2011.lsoacd.isin(ldn2011.lsoacd.values)]

# And save
ttw2011.to_csv(os.path.join(travel,'TTW-2011.csv'), index=False)

# Sanity check
print("2011 TTW data frame contains " + str(ttw2011.shape[0]) + " rows.")
ttw2011.sample(3, random_state=r_state)

## Cars & Vans

Tables:
- KS017
- KS404EW

In [ ]:
cv2001 = pd.read_csv(os.path.join(src,'2001','ks017.csv.gz'), 
                        header=5, skipfooter=4, skip_blank_lines=True, engine='python', compression='gzip')#, na_values="..")

# More analysis-friendly column names
cv2001.rename(columns={
    'super output areas - lower layer':'lsoanm',
    'mnemonic':'lsoacd',
    'All categories: Car or van availability':'total',
    'No cars or vans in household':'No vehicle',
    '1 car or van in household':'1 vehicle',
    '2 cars or vans in household':'2 vehicles',
    '3 cars or vans in household':'3 vehicles',
    '4 or more cars or vans in household':'4 or more vehicles',
    'sum of all cars or vans in the area':'Count of vehicles'
}, inplace=True)

# Don't need these
cv2001.drop(['lsoanm','Count of vehicles'], axis=1, inplace=True)

# Drop the rows we don't need
cv2001 = cv2001[cv2001.lsoacd.isin(ldn2004.lsoacd.values)]

# Save it
cv2001.to_csv(os.path.join(src,'Cars and Vans-2001.csv'), index=False)

# convert_to_2011(df,src,dest,nm)
convert_to_2011(cv2001, src, travel, 'Cars and Vans-2001.csv')

In [ ]:
cv2011 = pd.read_csv(os.path.join(src,'2011','ks404ew.csv.gz'), 
                        header=6, skipfooter=5, skip_blank_lines=True, engine='python', compression='gzip')#, na_values="..")

# More analysis-friendly column names
cv2011.rename(columns={
    '2011 super output area - lower layer':'lsoanm',
    'mnemonic':'lsoacd',
    'All categories: Car or van availability':'total',
    'No cars or vans in household':'No vehicle',
    '1 car or van in household':'1 vehicle',
    '2 cars or vans in household':'2 vehicles',
    '3 cars or vans in household':'3 vehicles',
    '4 or more cars or vans in household':'4 or more vehicles',
    'sum of all cars or vans in the area':'Count of vehicles'
}, inplace=True)

# Don't need these
cv2011.drop(['lsoanm','Count of vehicles'], axis=1, inplace=True)

# Drop the rows we don't need
cv2011 = cv2011[cv2011.lsoacd.isin(ldn2011.lsoacd.values)]

# Save it
cv2011.to_csv(os.path.join(travel,'Cars and Vans-2011.csv'), index=False)

# Sanity check, should be 4835
print("Have " + str(cv2011.shape[0]) + " rows of data.")
cv2011.sample(3, random_state=r_state)

## Age Structure

Tables:
- KS002
- KS102EW

In [ ]:
age2001 = pd.read_csv(os.path.join(src,'2001','ks002.csv.gz'), 
                        header=5, skipfooter=4, skip_blank_lines=True, engine='python', compression='gzip')#, na_values="..")

# More analysis-friendly column names
age2001.rename(columns={
    'super output areas - lower layer':'lsoanm',
    'mnemonic':'lsoacd',
    'All usual residents':'total'
}, inplace=True)

# Don't need these
age2001.drop(['lsoanm'], axis=1, inplace=True)
age2001 = age2001[age2001.lsoacd.isin(ldn2004.lsoacd.values)]

# Set index
age2001.set_index('lsoacd', inplace=True)

# Aggregate into a new data frame
age2001agg = pd.DataFrame()

# Drop the rows we don't need
age2001agg['Young Children'] = age2001[['Age 0 to 4','Age 5 to 7','Age 8 to 9']].sum(axis=1)
age2001agg['Teenagers'] = age2001[['Age 10 to 14','Age 15','Age 16 to 17']].sum(axis=1)
age2001agg['Young Adults'] = age2001[['Age 18 to 19','Age 20 to 24','Age 25 to 29']].sum(axis=1)
age2001agg['Adults'] = age2001[['Age 30 to 44','Age 45 to 59','Age 60 to 64']].sum(axis=1)
age2001agg['Retired'] = age2001[['Age 65 to 74','Age 75 to 84']].sum(axis=1)
age2001agg['Elderly'] = age2001[['Age 85 to 89','Age 90 and over']].sum(axis=1)
age2001agg['total'] = age2001[['total']]

# Save it
age2001agg.reset_index(inplace=True, drop=False)
age2001agg.to_csv(os.path.join(src,'Age Structure-2001.csv'), index=False)

# convert_to_2011(df,src,dest,nm)
convert_to_2011(age2001agg, src, household, 'Age Structure-2001.csv')

In [ ]:
age2011 = pd.read_csv(os.path.join(src,'2011','ks102ew.csv.gz'), 
                        header=6, skipfooter=5, skip_blank_lines=True, engine='python', compression='gzip')#, na_values="..")

# More analysis-friendly column names
age2011.rename(columns={
    '2011 super output area - lower layer':'lsoanm',
    'mnemonic':'lsoacd',
    'All usual residents':'total'
}, inplace=True)

# Don't need these
age2011.drop(['lsoanm'], axis=1, inplace=True)
age2011 = age2011[age2011.lsoacd.isin(ldn2011.lsoacd.values)]

# Set the index
age2011.set_index('lsoacd', inplace=True)

# Aggregate into a new data frame
age2011agg = pd.DataFrame()

# Drop the rows we don't need
age2011agg['Young Children'] = age2011[['Age 0 to 4','Age 5 to 7','Age 8 to 9']].sum(axis=1)
age2011agg['Teenagers'] = age2011[['Age 10 to 14','Age 15','Age 16 to 17']].sum(axis=1)
age2011agg['Young Adults'] = age2011[['Age 18 to 19','Age 20 to 24','Age 25 to 29']].sum(axis=1)
age2011agg['Adults'] = age2011[['Age 30 to 44','Age 45 to 59','Age 60 to 64']].sum(axis=1)
age2011agg['Retired'] = age2011[['Age 65 to 74','Age 75 to 84']].sum(axis=1)
age2011agg['Elderly'] = age2011[['Age 85 to 89','Age 90 and over']].sum(axis=1)
age2011agg['total'] = age2011[['total']]

# Save it
age2011agg.to_csv(os.path.join(household,'Age Structure-2011.csv'), index=True)

# Sanity check, should be 4835
print("Have " + str(age2011agg.shape[0]) + " rows of data.")
age2011agg.sample(3, random_state=r_state)

## Marital Status

Tables:
- KS004
- KS013EW

In [ ]:
mar2001 = pd.read_csv(os.path.join(src,'2001','ks004.csv.gz'), 
                        header=5, skipfooter=4, skip_blank_lines=True, engine='python', compression='gzip')#, na_values="..")

# More analysis-friendly column names
mar2001.rename(columns={
    'super output areas - lower layer':'lsoanm',
    'mnemonic':'lsoacd',
    'All people':'total',
    'Separated (but still legally married)':'Separated'
}, inplace=True)

# Aggregate these for compatibility with 2011
mar2001['Married'] = mar2001[['Married (first marriage)','Re-married']].sum(axis=1)

# Don't need these
mar2001.drop(['lsoanm','Married (first marriage)','Re-married'], axis=1, inplace=True)
mar2001 = mar2001[mar2001.lsoacd.isin(ldn2004.lsoacd.values)]

# Save it
mar2001.to_csv(os.path.join(src,'Marital Status-2001.csv'), index=False)

# convert_to_2011(df,src,dest,nm)
convert_to_2011(mar2001, src, household, 'Marital Status-2001.csv')

Note that transition from marriage to incorporate 'registered same-sex civil partnership'. It's hard to tell if these are directly comparable. I am assuming that the composition hasn't changed radically, but we'll need to test this carefully.

In [ ]:
mar2011 = pd.read_csv(os.path.join(src,'2011','ks103ew.csv.gz'), 
                        header=6, skipfooter=5, skip_blank_lines=True, engine='python', compression='gzip')#, na_values="..")

# More analysis-friendly column names
mar2011.rename(columns={
    '2011 super output area - lower layer':'lsoanm',
    'mnemonic':'lsoacd',
    'All usual residents aged 16+':'total',
    'Single (never married or never registered a same-sex civil partnership)':'Single (never married)',
    'Separated (but still legally married or still legally in a same-sex civil partnership)':'Separated',
    'Divorced or formerly in a same-sex civil partnership which is now legally dissolved':'Divorced',
    'Widowed or surviving partner from a same-sex civil partnership':'Widowed',
    'Married':'Married Temp'
}, inplace=True)

# Aggregate these for compatibility with rest of data
mar2011['Married'] = mar2011[['Married Temp','In a registered same-sex civil partnership']].sum(axis=1)

# Don't need these
mar2011.drop(['lsoanm','Married Temp','In a registered same-sex civil partnership'], axis=1, inplace=True)
mar2011 = mar2011[mar2011.lsoacd.isin(ldn2011.lsoacd.values)]

# Set the index
mar2011.set_index('lsoacd', inplace=True)

# Save it
mar2011.to_csv(os.path.join(household,'Marital Status-2011.csv'), index=True)

# Sanity check, should be 4835
print("Have " + str(mar2011.shape[0]) + " rows of data.")
mar2011.sample(3, random_state=r_state)

## Ethnicity

Tables:
- KS006
- KS201EW

In [ ]:
eth2001 = pd.read_csv(os.path.join(src,'2001','ks006.csv.gz'), 
                        header=5, skipfooter=4, skip_blank_lines=True, engine='python', compression='gzip')#, na_values="..")

# More analysis-friendly column names
eth2001.rename(columns={
    'super output areas - lower layer':'lsoanm',
    'mnemonic':'lsoacd',
    'All categories: Ethnic group':'total'
}, inplace=True)

# Don't need these
eth2001 = eth2001[eth2001.lsoacd.isin(ldn2004.lsoacd.values)]

# Set the index
eth2001.set_index('lsoacd', inplace=True)

# Aggregate these for compatibility with 2011
eth2001agg = pd.DataFrame()
eth2001agg['White']   = eth2001[[x for x in eth2001.columns if re.search('^White: ',x)]].sum(axis=1)
eth2001agg['Black']   = eth2001[[x for x in eth2001.columns if re.search('^Black',x)]].sum(axis=1)
eth2001agg['Asian']   = eth2001[[x for x in eth2001.columns if re.search('^Asian',x)]].sum(axis=1)
eth2001agg['Mixed']   = eth2001[[x for x in eth2001.columns if re.search('^Mixed: ',x)]].sum(axis=1)
eth2001agg['Other Ethnicity'] = eth2001[[x for x in eth2001.columns if re.search('Other: Chinese',x)]].sum(axis=1)
eth2001agg['Chinese'] = eth2001[[x for x in eth2001.columns if re.search('Other: Other',x)]].sum(axis=1)
eth2001agg['total']   = eth2001[['total']]

# Save it
eth2001agg.reset_index(inplace=True, drop=False)
eth2001agg.to_csv(os.path.join(src,'Ethnicity-2001.csv'), index=False)

# convert_to_2011(df,src,dest,nm)
convert_to_2011(eth2001agg, src, household, 'Ethnicity-2001.csv')

In [ ]:
eth2011 = pd.read_csv(os.path.join(src,'2011','ks201ew.csv.gz'), 
                        header=6, skipfooter=5, skip_blank_lines=True, engine='python', compression='gzip')#, na_values="..")

# More analysis-friendly column names
eth2011.rename(columns={
    '2011 super output area - lower layer':'lsoanm',
    'mnemonic':'lsoacd',
    'All usual residents':'total',
    'Mixed/multiple ethnic groups':'Mixed',
    'Black/African/Caribbean/Black British':'Black',
    'Other ethnic group':'Other Ethnicity'
}, inplace=True)

# Don't need these
eth2011 = eth2011[eth2011.lsoacd.isin(ldn2011.lsoacd.values)]

# Set the index
eth2011.set_index('lsoacd', inplace=True)

# Aggregate these for compatibility with rest of data
eth2011agg = pd.DataFrame()
eth2011agg = eth2011.loc[:,['White','Mixed','Black','Other Ethnicity']]
eth2011agg['Asian'] = eth2011[[x for x in eth2011.columns if re.search('^Asian/Asian British: [^C]',x)]].sum(axis=1)
eth2011agg['Chinese'] = eth2011[[x for x in eth2011.columns if re.search('Chinese$',x)]].sum(axis=1)
eth2011agg['total'] = eth2011[['total']]

# Save it
eth2011agg.to_csv(os.path.join(household,'Ethnicity-2011.csv'), index=True)

# Sanity check, should be 4835
print("Have " + str(eth2011agg.shape[0]) + " rows of data.")
eth2011agg.sample(3, random_state=r_state)

## Religion

Tables:
- KS007
- KS209EW

In [ ]:
rel2001 = pd.read_csv(os.path.join(src,'2001','ks007.csv.gz'), 
                        header=5, skipfooter=4, skip_blank_lines=True, engine='python', compression='gzip')#, na_values="..")

# More analysis-friendly column names
rel2001.rename(columns={
    'super output areas - lower layer':'lsoanm',
    'mnemonic':'lsoacd',
    'All categories: Religion':'total',
}, inplace=True)

# Don't need these
rel2001.drop(['lsoanm'], axis=1, inplace=True)
rel2001 = rel2001[rel2001.lsoacd.isin(ldn2004.lsoacd.values)]

# Save it
rel2001.to_csv(os.path.join(src,'Religion-2001.csv'), index=False)

# convert_to_2011(df,src,dest,nm)
convert_to_2011(rel2001, src, household, 'Religion-2001.csv')

In [ ]:
rel2011 = pd.read_csv(os.path.join(src,'2011','ks209ew.csv.gz'), 
                        header=6, skipfooter=5, skip_blank_lines=True, engine='python', compression='gzip')#, na_values="..")

# More analysis-friendly column names
rel2011.rename(columns={
    '2011 super output area - lower layer':'lsoanm',
    'mnemonic':'lsoacd',
    'All categories: Religion':'total',
}, inplace=True)

# Don't need these
rel2011.drop(['lsoanm'], axis=1, inplace=True)
rel2011 = rel2011[rel2011.lsoacd.isin(ldn2011.lsoacd.values)]

# Set the index
rel2011.set_index('lsoacd', inplace=True)

# Save it
rel2011.to_csv(os.path.join(household,'Religion-2011.csv'), index=True)

# Sanity check, should be 4835
print("Have " + str(rel2011.shape[0]) + " rows of data.")
rel2011.sample(3, random_state=r_state)

## Hours Worked

Tables:
- KS010
- KS604EW

In [ ]:
hrs2001 = pd.read_csv(os.path.join(src,'2001','ks010.csv.gz'), 
                        header=5, skipfooter=4, skip_blank_lines=True, engine='python', compression='gzip')#, na_values="..")

# More analysis-friendly column names
hrs2001.rename(columns={
    'super output areas - lower layer':'lsoanm',
    'mnemonic':'lsoacd'
}, inplace=True)

# Don't need these
hrs2001 = hrs2001[hrs2001.lsoacd.isin(ldn2004.lsoacd.values)]

# Aggregate these for compatibility with 2011
hrs2001['Males - Part-time - 15 hours or less worked'] = hrs2001[['Males - Part-time - 1 to 5 hours','Males - Part-time - 6 to 15 hours']].sum(axis=1)
hrs2001['Males - Full-time - 31 to 48 hours worked']   = hrs2001[['Males - Full-time - 31 to 37 hours','Males - Full-time - 38 to 48 hours']].sum(axis=1)
hrs2001['Females - Part-time - 15 hours or less worked'] = hrs2001[['Females - Part-time - 1 to 5 hours','Females - Part-time - 6 to 15 hours']].sum(axis=1)
hrs2001['Females - Full-time - 31 to 48 hours worked']   = hrs2001[['Females - Full-time - 31 to 37 hours','Females - Full-time - 38 to 48 hours']].sum(axis=1)
hrs2001['total']   = hrs2001[['All males aged 16-74 in employment','All females aged 16-74 in employment']].sum(axis=1)

hrs2001.drop(['lsoanm','Males - Part-time - 1 to 5 hours','Males - Part-time - 6 to 15 hours',
              'Males - Full-time - 31 to 37 hours','Males - Full-time - 38 to 48 hours',
              'Females - Part-time - 1 to 5 hours','Females - Part-time - 6 to 15 hours',
              'Females - Full-time - 31 to 37 hours','Females - Full-time - 38 to 48 hours',
              'All males aged 16-74 in employment','All females aged 16-74 in employment'], axis=1, inplace=True)

hrs2001.rename(columns=lambda x: re.sub(' worked$','',re.sub(' - Full-time - ',": ",re.sub(' - Part-time - ',": ",x))), inplace=True)
hrs2001.sample(3, random_state=r_state)

# Save it
hrs2001.to_csv(os.path.join(src,'Hours Worked-2001.csv'), index=False)

# convert_to_2011(df,src,dest,nm)
convert_to_2011(hrs2001, src, work, 'Hours Worked-2001.csv')

# Sanity check, should be 4835
print("Have " + str(pd.read_csv(os.path.join(work,'Hours Worked-2001.csv')).shape[0]) + " rows of data.")
pd.read_csv(os.path.join(work,'Hours Worked-2001.csv'), index_col=0).sample(3, random_state=r_state)

In [ ]:
hrs2011 = pd.read_csv(os.path.join(src,'2011','ks604ew.csv.gz'), 
                        header=6, skipfooter=5, skip_blank_lines=True, engine='python', compression='gzip')#, na_values="..")

# More analysis-friendly column names
hrs2011.rename(columns={
    '2011 super output area - lower layer':'lsoanm',
    'mnemonic':'lsoacd',
    'All usual residents aged 16 to 74 in employment the week before the census':'total',
}, inplace=True)

# Don't need these
hrs2011.drop(['lsoanm'], axis=1, inplace=True)
hrs2011 = hrs2011[hrs2011.lsoacd.isin(ldn2011.lsoacd.values)]

# Set the index
hrs2011.set_index('lsoacd', inplace=True)

# Rename cols for compatibility with 2001
hrs2011.rename(columns=lambda x: re.sub(' worked$','',re.sub('Full-time: ','',re.sub('Part-time: ','',x))), inplace=True)

# Save it
hrs2011.to_csv(os.path.join(work,'Hours Worked-2011.csv'), index=True)

# Sanity check, should be 4835
print("Have " + str(hrs2011.shape[0]) + " rows of data.")
hrs2011.sample(3, random_state=r_state)

## Industry

Real compatibilty issues here: [this](http://www.businessballs.com/industrialclassifications.htm) is the best comparison I can find to use for a consistent mapping.

Tables:
- KS011a
- KS607EW

**ALSO**: note that you can't use a comma even in the header of the CSV file since GeoConvert can't cope.

In [ ]:
ind2001 = pd.read_csv(os.path.join(src,'2001','ks011a.csv.gz'), 
                        header=5, skipfooter=4, skip_blank_lines=True, engine='python', compression='gzip')#, na_values="..")
# More analysis-friendly column names
ind2001.rename(columns={
    'super output areas - lower layer':'lsoanm',
    'mnemonic':'lsoacd',
    'All categories: Industry':'total'
}, inplace=True)

# Don't need these
ind2001.drop(['lsoanm'], axis=1, inplace=True)
ind2001 = ind2001[ind2001.lsoacd.isin(ldn2004.lsoacd.values)]

# Set the index
ind2001.set_index('lsoacd', inplace=True)

# Aggregate/rename as needed for compatibility
ind2001agg = pd.DataFrame()
ind2001agg['A. Agriculture et al'] = ind2001[[x for x in ind2001.columns if re.search('^[AB] ',x)]].sum(axis=1)
ind2001agg['C. Mining'] = ind2001[[x for x in ind2001.columns if re.search('^[C] ',x)]].sum(axis=1)
ind2001agg['D. Manufacturing'] = ind2001[[x for x in ind2001.columns if re.search('^[D] ',x)]].sum(axis=1)
ind2001agg['E. Utilities'] = ind2001[[x for x in ind2001.columns if re.search('^[E] ',x)]].sum(axis=1)
ind2001agg['F. Construction'] = ind2001[[x for x in ind2001.columns if re.search('^[F] ',x)]].sum(axis=1)
ind2001agg['G. Wholesale and retail'] = ind2001[[x for x in ind2001.columns if re.search('^[G] ',x)]].sum(axis=1)
ind2001agg['H. Hotels and restaurants'] = ind2001[[x for x in ind2001.columns if re.search('^[H] ',x)]].sum(axis=1)
ind2001agg['I. Transport storage and comms'] = ind2001[[x for x in ind2001.columns if re.search('^[I] ',x)]].sum(axis=1)
ind2001agg['J. Financial'] = ind2001[[x for x in ind2001.columns if re.search('^[J] ',x)]].sum(axis=1)
ind2001agg['K. Professional'] = ind2001[[x for x in ind2001.columns if re.search('^[K] ',x)]].sum(axis=1)
ind2001agg['L. Public Sector'] = ind2001[[x for x in ind2001.columns if re.search('^[L] ',x)]].sum(axis=1)
ind2001agg['M. Education'] = ind2001[[x for x in ind2001.columns if re.search('^[M] ',x)]].sum(axis=1)
ind2001agg['N. Health and social work'] = ind2001[[x for x in ind2001.columns if re.search('^[N] ',x)]].sum(axis=1)
ind2001agg['P. All Other'] = ind2001[[x for x in ind2001.columns if re.search('^[O],',x)]].sum(axis=1)
ind2001agg['total'] = ind2001[['total']]

# Save it
ind2001agg.reset_index(inplace=True, drop=False)
ind2001agg.to_csv(os.path.join(src,'Industry-2001.csv'), index=False)

# convert_to_2011(df,src,dest,nm)
convert_to_2011(ind2001agg, src, work, 'Industry-2001.csv')

In [ ]:
ind2011 = pd.read_csv(os.path.join(src,'2011','ks607ew.csv.gz'), 
                        header=7, skipfooter=5, skip_blank_lines=True, engine='python', compression='gzip')#, na_values="..")
ind2011.rename(columns={
    '2011 super output area - lower layer':'lsoanm',
    'mnemonic':'lsoacd',
    'All categories: Industry':'total'
}, inplace=True)

# Don't need these
ind2011.drop(['lsoanm'], axis=1, inplace=True)
ind2011 = ind2011[ind2011.lsoacd.isin(ldn2011.lsoacd.values)]

# Set the index
ind2011.set_index('lsoacd', inplace=True)

# Aggregate/rename as needed for compatibility
ind2011agg = pd.DataFrame()
ind2011agg['A. Agriculture et al'] = ind2011[[x for x in ind2011.columns if re.search('^[A] ',x)]].sum(axis=1)
ind2011agg['C. Mining'] = ind2011[[x for x in ind2011.columns if re.search('^[B] ',x)]].sum(axis=1)
ind2011agg['D. Manufacturing'] = ind2011[[x for x in ind2011.columns if re.search('^[C] ',x)]].sum(axis=1)
ind2011agg['E. Utilities'] = ind2011[[x for x in ind2011.columns if re.search('^[DE] ',x)]].sum(axis=1)
ind2011agg['F. Construction'] = ind2011[[x for x in ind2011.columns if re.search('^[F] ',x)]].sum(axis=1)
ind2011agg['G. Wholesale and retail'] = ind2011[[x for x in ind2011.columns if re.search('^[G] ',x)]].sum(axis=1)
ind2011agg['H. Hotels and restaurants'] = ind2011[[x for x in ind2011.columns if re.search('^[I] ',x)]].sum(axis=1)
ind2011agg['I. Transport storage and comms'] = ind2011[[x for x in ind2011.columns if re.search('^[HJ] ',x)]].sum(axis=1)
ind2011agg['J. Financial'] = ind2011[[x for x in ind2011.columns if re.search('^[K] ',x)]].sum(axis=1)
ind2011agg['K. Professional'] = ind2011[[x for x in ind2011.columns if re.search('^[LMN] ',x)]].sum(axis=1)
ind2011agg['L. Public Sector'] = ind2011[[x for x in ind2011.columns if re.search('^[O] ',x)]].sum(axis=1)
ind2011agg['M. Education'] = ind2011[[x for x in ind2011.columns if re.search('^[P] ',x)]].sum(axis=1)
ind2011agg['N. Health and social work'] = ind2011[[x for x in ind2011.columns if re.search('^[Q] ',x)]].sum(axis=1)
ind2011agg['P. All Other'] = ind2011[[x for x in ind2011.columns if re.search('^[RSTU],',x)]].sum(axis=1)
ind2011agg['total'] = ind2011[['total']]

# Save it
ind2011agg.to_csv(os.path.join(work,'Industry-2011.csv'), index=True)

# Sanity check, should be 4835
print("Have " + str(ind2011agg.shape[0]) + " rows of data.")
ind2011agg.sample(3, random_state=r_state)

## Tenure

Tables:
- KS018
- KS402EW

In [ ]:
ten2001 = pd.read_csv(os.path.join(src,'2001','ks018.csv.gz'), 
                        header=5, skipfooter=4, skip_blank_lines=True, engine='python', compression='gzip')#, na_values="..")

# More analysis-friendly column names
ten2001.rename(columns={
    'super output areas - lower layer':'lsoanm',
    'mnemonic':'lsoacd',
    'All categories: Tenure':'total',
    'Owned outright':'Owned: Outright',
    'Owned with a mortgage or loan':'Owned: Mortgaged',
    'Rented from council(local authority)':'Rented: Council',
    'Rented from a housing association/registered social landlord':'Rented: HA or RSL',
    'Rented from a private landlord or letting agency':'Rented: Private',
    'Other':'Other Tenure Type'
}, inplace=True)

# Don't need these
ten2001 = ten2001[ten2001.lsoacd.isin(ldn2004.lsoacd.values)]
ten2001.drop(['lsoanm'], axis=1, inplace=True)

# Save it
ten2001.to_csv(os.path.join(src,'Tenure-2001.csv'), index=False)

# convert_to_2011(df,src,dest,nm)
convert_to_2011(ten2001, src, housing, 'Tenure-2001.csv')

In [ ]:
ten2011 = pd.read_csv(os.path.join(src,'2011','ks402ew.csv.gz'), 
                        header=6, skipfooter=5, skip_blank_lines=True, engine='python', compression='gzip')#, na_values="..")

ten2011.drop(['Owned','Social rented','Private rented'], axis=1, inplace=True)

# More analysis-friendly column names
ten2011.rename(columns={
    '2011 super output area - lower layer':'lsoanm',
    'mnemonic':'lsoacd',
    'All households':'total',
    'Owned: Owned outright':'Owned: Outright',
    'Owned: Owned with a mortgage or loan':'Owned: Mortgaged',
    'Shared ownership (part owned and part rented)':'Shared ownership',
    'Social rented: Rented from council (Local Authority)':'Rented: Council',
    'Social rented: Other':'Rented: HA or RSL',
    'Private rented: Private landlord or letting agency':'Rented: Private'
}, inplace=True)

# Don't need these
ten2011.drop(['lsoanm'], axis=1, inplace=True)
ten2011 = ten2011[ten2011.lsoacd.isin(ldn2011.lsoacd.values)]

# Set the index
ten2011.set_index('lsoacd', inplace=True)

# Aggregate to match 2001 data
ten2011['Other Tenure Type'] = ten2011[['Private rented: Other','Living rent free']].sum(axis=1)
ten2011.drop(['Private rented: Other','Living rent free'], axis=1, inplace=True)

# Save it
ten2011.to_csv(os.path.join(housing,'Tenure-2011.csv'), index=True)

# Sanity check, should be 4835
print("Have " + str(ten2011.shape[0]) + " rows of data.")
ten2011.sample(3, random_state=r_state)

## Household Composition

Tables:
- KS020
- KS105EW

In [ ]:
hhc2001 = pd.read_csv(os.path.join(src,'2001','ks020.csv.gz'), 
                        header=5, skipfooter=4, skip_blank_lines=True, engine='python', compression='gzip')#, na_values="..")

# More analysis-friendly column names
hhc2001.rename(columns={
    'super output areas - lower layer':'lsoanm',
    'mnemonic':'lsoacd',
    'All households':'total'
}, inplace=True)
hhc2001.rename(columns=lambda x: re.sub('One person household(?: - )?','1P: ',x), inplace=True)
hhc2001.rename(columns=lambda x: re.sub('One family and no others(?: - )?','1F: ',x), inplace=True)
hhc2001.rename(columns=lambda x: re.sub('One person household(?: - )?','1P: ',x), inplace=True)
hhc2001.rename(columns=lambda x: re.sub('One family only(?: - )?','1F: ',x), inplace=True)
hhc2001.rename(columns=lambda x: re.sub(': married couple households(?: - )?','M: ',x), inplace=True)
hhc2001.rename(columns=lambda x: re.sub(': cohabiting couple households(?: - )?','C: ',x), inplace=True)
hhc2001.rename(columns=lambda x: re.sub(': Lone parent households(?: - )?','L: ',x), inplace=True)
hhc2001.rename(columns=lambda x: re.sub('Other households(?: - )?','O: ',x), inplace=True)
hhc2001.rename(columns=lambda x: re.sub('(?:All p|P)ensioners?','65+',x), inplace=True)
hhc2001.rename(columns=lambda x: re.sub('All students','Students',x), inplace=True)
hhc2001.rename(columns=lambda x: re.sub('with dependent children','Dependent children',x), inplace=True)
hhc2001.rename(columns=lambda x: re.sub('no children','No children',x), inplace=True)
hhc2001.rename(columns=lambda x: re.sub('all children non-dependent','All children non-dependent',x), inplace=True)

# Don't need these
hhc2001 = hhc2001[hhc2001.lsoacd.isin(ldn2004.lsoacd.values)]
hhc2001.drop(['lsoanm'], axis=1, inplace=True)

# Save it
hhc2001.to_csv(os.path.join(src,'Household Composition-2001.csv'), index=False)

# convert_to_2011(df,src,dest,nm)
convert_to_2011(hhc2001, src, household, 'Household Composition-2001.csv')

In [ ]:
hhc2011 = pd.read_csv(os.path.join(src,'2011','ks105ew.csv.gz'), 
                        header=6, skipfooter=5, skip_blank_lines=True, engine='python', compression='gzip')#, na_values="..")

# More analysis-friendly column names
hhc2011.rename(columns={
    '2011 super output area - lower layer':'lsoanm',
    'mnemonic':'lsoacd',
    'All categories: Household composition':'total'
}, inplace=True)
hhc2011.rename(columns=lambda x: re.sub('One person household(?:\: )?','1P: ',x), inplace=True)
hhc2011.rename(columns=lambda x: re.sub('One family only(?:\: )?','1F: ',x), inplace=True)
hhc2011.rename(columns=lambda x: re.sub(': Married or same-sex civil partnership couple(?:\: )?','M: ',x), inplace=True)
hhc2011.rename(columns=lambda x: re.sub(': Cohabiting couple(?:\: )?','C: ',x), inplace=True)
hhc2011.rename(columns=lambda x: re.sub(': Lone parent(?:\: )?','L: ',x), inplace=True)
hhc2011.rename(columns=lambda x: re.sub('Other household types(?:\: )?','O: ',x), inplace=True)
hhc2011.rename(columns=lambda x: re.sub('A(?:ll a)?ged 65 and over','65+',x), inplace=True)
hhc2011.rename(columns=lambda x: re.sub('All full-time students','Students',x), inplace=True)

# Don't need these
hhc2011.drop(['lsoanm','One family household','1P: ','1FM: ','1FC: ','1FL: ','O: '], axis=1, inplace=True)
hhc2011 = hhc2011[hhc2011.lsoacd.isin(ldn2011.lsoacd.values)]

# Set the index
hhc2011.set_index('lsoacd', inplace=True)

# Save it
hhc2011.to_csv(os.path.join(household,'Household Composition-2011.csv'), index=True)

# Sanity check, should be 4835
print("Have " + str(hhc2011.shape[0]) + " rows of data.")
hhc2011.sample(3, random_state=r_state)

## Population Density

Tables:
- UV002
- QS102EW

In [ ]:
den2001 = pd.read_csv(os.path.join(src,'2001','uv002.csv.gz'), 
                        header=5, skipfooter=4, skip_blank_lines=True, engine='python', compression='gzip')#, na_values="..")

# More analysis-friendly column names
den2001.rename(columns={
    'super output areas - lower layer':'lsoanm',
    'mnemonic':'lsoacd',
    'Density (number of persons per hectare)':'Density'
}, inplace=True)

# Don't need these
den2001 = den2001[den2001.lsoacd.isin(ldn2004.lsoacd.values)]
den2001.drop(['lsoanm','All usual residents','Area Hectares'], axis=1, inplace=True)

# Save it
den2001.to_csv(os.path.join(src,'Density-2001.csv'), index=False)

# convert_to_2011(df,src,dest,nm)
convert_to_2011(den2001, src, housing, 'Density-2001.csv')

In [ ]:
den2011 = pd.read_csv(os.path.join(src,'2011','qs102ew.csv.gz'), 
                        header=6, skipfooter=5, skip_blank_lines=True, engine='python', compression='gzip')#, na_values="..")

# More analysis-friendly column names
den2011.rename(columns={
    '2011 super output area - lower layer':'lsoanm',
    'mnemonic':'lsoacd',
    'Density (number of persons per hectare)':'Density'
}, inplace=True)

# Don't need these
den2011.drop(['lsoanm','All usual residents','Area Hectares'], axis=1, inplace=True)
den2011 = den2011[den2011.lsoacd.isin(ldn2011.lsoacd.values)]

# Set the index
den2011.set_index('lsoacd', inplace=True)

# Save it
den2011.to_csv(os.path.join(housing,'Density-2011.csv'), index=True)

# Sanity check, should be 4835
print("Have " + str(den2011.shape[0]) + " rows of data.")
den2011.sample(3, random_state=r_state)

## Dependent Children

Tables:
- UV006
- QS118EW

In [ ]:
dep2001 = pd.read_csv(os.path.join(src,'2001','uv006.csv.gz'), 
                        header=5, skipfooter=4, skip_blank_lines=True, engine='python', compression='gzip')#, na_values="..")

# More analysis-friendly column names
dep2001.rename(columns={
    'super output areas - lower layer':'lsoanm',
    'mnemonic':'lsoacd',
    'All families in households':'total',
    'No dependent children in family':'0D'
}, inplace=True)

# Don't need these
dep2001 = dep2001[dep2001.lsoacd.isin(ldn2004.lsoacd.values)]
dep2001.drop(['lsoanm'], axis=1, inplace=True)

# Tidy up column names
dep2001.rename(columns=lambda x: re.sub('One dependent child in family aged ?','1D: ',x), inplace=True)
dep2001.rename(columns=lambda x: re.sub('Two dependent children in family; youngest aged','2D: ',x), inplace=True)

# Save it
dep2001.to_csv(os.path.join(src,'Dependent Children-2001.csv'), index=False)

# convert_to_2011(df,src,dest,nm)
convert_to_2011(dep2001, src, household, 'Dependent Children-2001.csv')

In [ ]:
dep2011 = pd.read_csv(os.path.join(src,'2011','qs118ew.csv.gz'), 
                        header=6, skipfooter=5, skip_blank_lines=True, engine='python', compression='gzip')#, na_values="..")

# More analysis-friendly column names
dep2011.rename(columns={
    '2011 super output area - lower layer':'lsoanm',
    'mnemonic':'lsoacd',
    'All families in households':'total',
    'No dependent children in family':'0D'
}, inplace=True)

# Don't need these or they are incompatible with 2001 data
dep2011.drop(['lsoanm',
              'Three or more dependent children in family; youngest aged 0 to 4',
              'Three or more dependent children in family; youngest aged 5 to 11',
              'Three or more dependent children in family; youngest aged 12 to 18',
              'Total dependent children'], axis=1, inplace=True)
dep2011 = dep2011[dep2011.lsoacd.isin(ldn2011.lsoacd.values)]

# Set the index
dep2011.set_index('lsoacd', inplace=True)

# Tidy up column names
dep2011.rename(columns=lambda x: re.sub('One dependent child in family aged ?','1D: ',x), inplace=True)
dep2011.rename(columns=lambda x: re.sub('Two dependent children in family; youngest aged','2D: ',x), inplace=True)

# Save it
dep2011.to_csv(os.path.join(household,'Dependent Children-2011.csv'), index=True)

# Sanity check, should be 4835
print("Have " + str(dep2011.shape[0]) + " rows of data.")
dep2011.sample(3, random_state=r_state)

## Country of Birth

Tables:
- UV008
- QS203EW

These are hard to work out because the categories change a lot, but my best attempt at mapping between them is:
- England
- Scotland
- Wales
- **Rest of UK** (UK _minus_ the above)
- EU 2001 Members
- Poland
- Turkey
- **Rest of Europe** (Bits of Europe not covered above)
- North Africa
- South and Eastern Africa
- **Rest of Africa** (Africa _minus_ the above)
- Middle East
- Far East / Eastern Asia
- South Asia / Souther Asia
- **Rest of Asia** (Asia _minus_ the above)

For reasons that I _cannot_ work out I get a sum of groups that is greater than the total. Since every LSOA is affected the same way this shouldn't matter but clearly I am double-counting _someone_ in there.

In [ ]:
cb2001 = pd.read_csv(os.path.join(src,'2001','uv008.csv.gz'), 
                        header=5, skipfooter=4, skip_blank_lines=True, engine='python', compression='gzip')#, na_values="..")

# More analysis-friendly column names
cb2001.rename(columns={
    'super output areas - lower layer':'lsoanm',
    'mnemonic':'lsoacd',
    'All categories: Country of birth':'total',
    'EU countries':'EU 2001 Members',
    'Republic of Ireland':'Ireland'
}, inplace=True)

# Don't need these
cb2001 = cb2001[cb2001.lsoacd.isin(ldn2004.lsoacd.values)]

# Set index
cb2001.set_index('lsoacd', inplace=True)

# Aggregate for compatibility
cb2001agg = cb2001.loc[:,['total', 'England']]

cb2001agg['Rest of UK'] = cb2001.loc[:,'United Kingdom'] \
        - cb2001.loc[:,'England']

cb2001agg['EU 2001 Members'] = cb2001.loc[:,'EU 2001 Members'] \
        + cb2001.loc[:,'Ireland']

cb2001agg['Rest of Europe'] = cb2001.loc[:,'Europe'] \
        - (cb2001.loc[:,'United Kingdom'] \
           + cb2001.loc[:,'EU 2001 Members'] \
           + cb2001.loc[:,'Ireland'])
        
cb2001agg['Asia'] = cb2001.loc[:,'Asia']
cb2001agg['Africa'] = cb2001.loc[:,'Africa']
cb2001agg['Oceania'] = cb2001.loc[:,'Oceania']

cb2001agg['Americas'] = cb2001[['North America','South America']].sum(axis=1)

cb2001agg.reset_index(inplace=True, drop=False)
    
# Save it
cb2001agg.to_csv(os.path.join(src,'Country of Birth-2001.csv'), index=False)

# convert_to_2011(df,src,dest,nm)
convert_to_2011(cb2001agg, src, household, 'Country of Birth-2001.csv')

In [ ]:
cb2011 = pd.read_csv(os.path.join(src,'2011','qs203ew.csv.gz'), 
                        header=6, skipfooter=5, skip_blank_lines=True, engine='python', compression='gzip')#, na_values="..")

# More analysis-friendly column names
cb2011.rename(columns={
    '2011 super output area - lower layer':'lsoanm',
    'mnemonic':'lsoacd',
    'All categories: Country of birth':'total',
    'Europe: Total':'Europe',
    'Europe: United Kingdom: Total':'United Kingdom',
    'Africa: Total':'Africa',
    'Middle East and Asia: Total':'Asia',
    'The Americas and the Caribbean: Total':'Americas',
    'Antarctica and Oceania: Total':'Oceania',
    'Europe: Other Europe: EU countries: Member countries in March 2001: Total':'EU 2001 Members'
}, inplace=True)

# Don't need these or they are incompatible with 2001 data
cb2011 = cb2011[cb2011.lsoacd.isin(ldn2011.lsoacd.values)]

cb2011.rename(columns=lambda x: re.sub('^(?:Europe|Africa|Middle East and Asia|The Americas and the Caribbean|Antarctica and Oceania): ','',x), inplace=True)
cb2011.rename(columns=lambda x: re.sub('^(?:Other Europe|United Kingdom): ','',x), inplace=True)
cb2011.rename(columns=lambda x: re.sub(': Total$','',x), inplace=True)

# Set the index
cb2011.set_index('lsoacd', inplace=True)

cb2011agg = cb2011.loc[:,['total', 'England']]

cb2011agg['Rest of UK'] = cb2011.loc[:,'United Kingdom'] \
        - cb2011.loc[:,'England']

cb2011agg['EU 2001 Members'] = cb2011.loc[:,'EU 2001 Members'] \
        + cb2011.loc[:,'Ireland']
    
cb2011agg['Rest of Europe'] = cb2011.loc[:,'Europe'] \
        - (cb2011.loc[:,'United Kingdom'] \
           + cb2011.loc[:,'EU 2001 Members'] \
           + cb2011.loc[:,'Ireland'])

cb2011agg['Asia'] = cb2011.loc[:,'Asia']
cb2011agg['Africa'] = cb2011.loc[:,'Africa']
cb2011agg['Oceania'] = cb2011.loc[:,'Oceania']
cb2011agg['Americas'] = cb2011.loc[:,'Americas']

# Save it
cb2011agg.to_csv(os.path.join(household,'Country of Birth-2011.csv'), index=True)

# Sanity check, should be 4835
print("Have " + str(cb2011agg.shape[0]) + " rows of data.")
cb2011agg.sample(3, random_state=r_state)

## General Health

Tables:
- UV020
- QS302EW

In [ ]:
gh2001 = pd.read_csv(os.path.join(src,'2001','uv020.csv.gz'), 
                        header=5, skipfooter=4, skip_blank_lines=True, engine='python', compression='gzip')#, na_values="..")

# More analysis-friendly column names
gh2001.rename(columns={
    'super output areas - lower layer':'lsoanm',
    'mnemonic':'lsoacd',
    'Fairly good health':'Fair health',
    'Not good health':'Poor health'
}, inplace=True)

# Don't need these
gh2001 = gh2001[gh2001.lsoacd.isin(ldn2004.lsoacd.values)]
gh2001.drop(['lsoanm'], axis=1, inplace=True)

# Save it
gh2001.to_csv(os.path.join(src,'General Health-2001.csv'), index=False)

# convert_to_2011(df,src,dest,nm)
convert_to_2011(gh2001, src, household, 'General Health-2001.csv')

In [ ]:
gh2011 = pd.read_csv(os.path.join(src,'2011','qs302ew.csv.gz'), 
                        header=6, skipfooter=5, skip_blank_lines=True, engine='python', compression='gzip')#, na_values="..")

# More analysis-friendly column names
gh2011.rename(columns={
    '2011 super output area - lower layer':'lsoanm',
    'mnemonic':'lsoacd',
    'All categories: General health':'total'
}, inplace=True)

# Don't need these or they are incompatible with 2001 data
gh2011 = gh2011[gh2011.lsoacd.isin(ldn2011.lsoacd.values)]

# Set the index
gh2011.set_index('lsoacd', inplace=True)

gh2011['Good health'] = gh2011[['Very good health','Good health']].sum(axis=1)
gh2011['Poor health'] = gh2011[['Bad health','Very bad health']].sum(axis=1)

gh2011.drop(['lsoanm','Very good health',
             'Bad health','Very bad health'], axis=1, inplace=True)

# Save it
gh2011.to_csv(os.path.join(household,'General Health-2011.csv'), index=True)

# Sanity check, should be 4835
print("Have " + str(gh2011.shape[0]) + " rows of data.")
gh2011.sample(3, random_state=r_state)

## NS-SeC

This is a tricky one since the categories are [not considered directly comparable](https://www.ons.gov.uk/employmentandlabourmarket/peopleinwork/employmentandemployeetypes/bulletins/keystatisticsandquickstatisticsforlocalauthoritiesintheunitedkingdom/2013-12-04) across Census years because of movement between classifications (e.g. an occupation that moves from being unskilled to skilled), but it's also a very useful composite view.

Tables:
- UV031
- QS607EW

In [ ]:
nssec2001 = pd.read_csv(os.path.join(src,'2001','uv031.csv.gz'), 
                        header=5, skipfooter=4, skip_blank_lines=True, engine='python', compression='gzip')#, na_values="..")

# More analysis-friendly column names
nssec2001.rename(columns={
    'super output areas - lower layer':'lsoanm',
    'mnemonic':'lsoacd',
    'All categories: NS-SeC':'total',
    '2. Lower managerial, administrative and professional occupations':'2. Lower managerial and professional occupations' # Avoid commas in columns
}, inplace=True)

# Don't need these
nssec2001 = nssec2001[nssec2001.lsoacd.isin(ldn2004.lsoacd.values)]
nssec2001.drop(['lsoanm','1. Higher managerial, administrative and professional occupations'], axis=1, inplace=True)
nssec2001.drop([x for x in nssec2001.columns if x.startswith("L")], axis=1, inplace=True)

# Save it
nssec2001.to_csv(os.path.join(src,'NS-SeC-2001.csv'), index=False)

# convert_to_2011(df,src,dest,nm)
convert_to_2011(nssec2001, src, work, 'NS-SeC-2001.csv')

In [ ]:
nssec2011 = pd.read_csv(os.path.join(src,'2011','qs607ew.csv.gz'), 
                        header=6, skipfooter=5, skip_blank_lines=True, engine='python', compression='gzip')#, na_values="..")

# More analysis-friendly column names
nssec2011.rename(columns={
    '2011 super output area - lower layer':'lsoanm',
    'mnemonic':'lsoacd',
    'All categories: NS-SeC':'total',
    '2. Lower managerial, administrative and professional occupations':'2. Lower managerial and professional occupations' # Avoid commas in columns
}, inplace=True)

# Don't need these or they are incompatible with 2001 data
nssec2011 = nssec2011[nssec2011.lsoacd.isin(ldn2011.lsoacd.values)]
nssec2011.drop(['lsoanm','1. Higher managerial, administrative and professional occupations'], axis=1, inplace=True)
nssec2011.drop([x for x in nssec2011.columns if x.startswith("L")], axis=1, inplace=True)

# Set the index
nssec2011.set_index('lsoacd', inplace=True)

# Save it
nssec2011.to_csv(os.path.join(work,'NS-SeC-2011.csv'), index=True)

# Sanity check, should be 4835
print("Have " + str(nssec2011.shape[0]) + " rows of data.")
nssec2011.sample(3, random_state=r_state)

## Economic Activity

Tables:
- UV028
- QS601EW

In [ ]:
ea2001 = pd.read_csv(os.path.join(src,'2001','uv028.csv.gz'), 
                        header=5, skipfooter=4, skip_blank_lines=True, engine='python', compression='gzip')#, na_values="..")

# More analysis-friendly column names
ea2001.rename(columns={
    'super output areas - lower layer':'lsoanm',
    'mnemonic':'lsoacd',
    'All categories: Economic activity':'total',
    'Employee: Part-time':'PT Employee',
    'Employee: Full-time':'FT Employee',
    'Self-employed with employees: Part-time':'PT Self-Employed with Employees',
    'Self-employed with employees: Full-time':'FT Self-Employed with Employees',
    'Self-employed without employees: Part-time':'PT Self-Employed without Employees',
    'Self-employed without employees: Full-time':'FT Self-Employed without Employees',
    'Economically inactive: Total':'Economically inactive'
}, inplace=True)

# Don't need these
ea2001 = ea2001[ea2001.lsoacd.isin(ldn2004.lsoacd.values)]
ea2001.drop('lsoanm', inplace=True, axis=1)

# Inactive less Retired and Looking after home or family
ea2001['Economically inactive'] = ea2001.loc[:,'Economically inactive'] \
    - (ea2001.loc[:,'Retired'] + 
       ea2001.loc[:,'Looking after home or family'])

# Save it
ea2001.to_csv(os.path.join(src,'Economic Activity-2001.csv'), index=False)

# convert_to_2011(df,src,dest,nm)
convert_to_2011(ea2001, src, work, 'Economic Activity-2001.csv')

In [ ]:
ea2011 = pd.read_csv(os.path.join(src,'2011','qs601ew.csv.gz'), 
                        header=6, skipfooter=5, skip_blank_lines=True, engine='python', compression='gzip')#, na_values="..")

# More analysis-friendly column names
ea2011.rename(columns=lambda x: re.sub('Economically (?:in)?active: ','',x), inplace=True)
ea2011.rename(columns={
    '2011 super output area - lower layer':'lsoanm',
    'mnemonic':'lsoacd',
    'All categories: Economic activity':'total',
    'Employee: Part-time':'PT Employee',
    'Employee: Full-time':'FT Employee',
    'Self-employed with employees: Part-time':'PT Self-Employed with Employees',
    'Self-employed with employees: Full-time':'FT Self-Employed with Employees',
    'Self-employed without employees: Part-time':'PT Self-Employed without Employees',
    'Self-employed without employees: Full-time':'FT Self-Employed without Employees',
    'Total':'Economically inactive'
}, inplace=True)

# Don't need these or they are incompatible with 2001 data
ea2011 = ea2011[ea2011.lsoacd.isin(ldn2011.lsoacd.values)]
ea2011.drop(['lsoanm'], axis=1, inplace=True)

# Inactive less Retired and Looking after home or family
ea2001['Economically inactive'] = ea2001.loc[:,'Economically inactive'] \
    - (ea2001.loc[:,'Retired'] + 
       ea2001.loc[:,'Looking after home or family'])

# Set the index
ea2011.set_index('lsoacd', inplace=True)

# Save it
ea2011.to_csv(os.path.join(work,'Economic Activity-2011.csv'), index=True)

# Sanity check, should be 4835
print("Have " + str(ea2011.shape[0]) + " rows of data.")
ea2011.sample(3, random_state=r_state)